# ЛР2
Абельдинов Рафаэль

## Задание
---
Сформировать отчет с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языкаов прогарммирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчет сохранить в формате Apache Parquet.

In [ ]:
import os
import sys
from pyspark.sql import Row
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [ ]:
# Задаем переменные окружения для корректной работы
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.17.0 pyspark-shell'

spark = SparkSession.builder.getOrCreate()
spark

In [ ]:
# Скачиваем датасеты из репозитория
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv

--2025-03-26 09:42:28--  https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74162295 (71M) [text/plain]
Saving to: ‘posts_sample.xml.5’

posts_sample.xml.5   98%[==================> ]  69.48M  --.-KB/s    eta 22s    ^C
--2025-03-26 10:03:07--  https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40269 (39K) [text/plain]
Saving to: ‘programming-languages.csv.4’

programming-languag 100%[===================>]  39.33K   132KB/s    in 0.3s    

2025-03-26 10:03:08 (132 KB/s) - ‘programming-languages.csv.4’ saved [40269/40269]



In [ ]:
# Загружаем датасеты с помощью Spark
postsData = spark.read.format('xml').option('rowTag', 'row').option("timestampFormat", 'y/M/d H:m:s').load('posts_sample.xml')
langData = spark.read.format('csv').option('header', 'true').option("inferSchema", True).load('programming-languages.csv').dropna()

In [ ]:
print("Schema: ")
postsData.printSchema()
print("\n\nData:")
postsData.show(n = 2)
print("\n\nSummary:")
postsData.describe().show()

Schema: 
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)



Data:
+-----------------+------------+--------------------+-------

In [ ]:
print("Schema: ")
langData.printSchema()
print("\n\nData:")
langData.show(n = 2)
print("\n\nSummary:")
langData.describe().show()

Schema: 
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)



Data:
+----------+--------------------+
|      name|       wikipedia_url|
+----------+--------------------+
|   A# .NET|https://en.wikipe...|
|A# (Axiom)|https://en.wikipe...|
+----------+--------------------+
only showing top 2 rows



Summary:
+-------+--------+--------------------+
|summary|    name|       wikipedia_url|
+-------+--------+--------------------+
|  count|     699|                 699|
|   mean|    NULL|                NULL|
| stddev|    NULL|                NULL|
|    min|@Formula|https://en.wikipe...|
|    max|xHarbour|https://en.wikipe...|
+-------+--------+--------------------+



In [ ]:
# Отбираем посты в необходимом периоде
posts_in_period = postsData.filter(F.col("_CreationDate").between(*("2010-01-01",  "2020-12-31")))

In [ ]:
# Отбираем названия языков программирования
lang_names = [str(x[0]) for x in langData.collect()]
lang_names[:10]

['A# .NET',
 'A# (Axiom)',
 'A-0 System',
 'A+',
 'A++',
 'ABAP',
 'ABC',
 'ABC ALGOL',
 'ABSET',
 'ABSYS']

In [ ]:
# Функция формирования тега языка программирования для поста
def include_name(x):
    tag = None
    for name in lang_names:
        n = '<' + name.lower() + '>'
        if n in str(x._Tags).lower():
            tag = name
            break
    if tag is None:
        tag = 'No'

    return (x[6], tag)


In [ ]:
# Применение функции к данным и их фильтрация
posts_in_period_ = posts_in_period.rdd.map(include_name).filter(lambda x: x[1] != 'No')

# Группировка данных, подсчет количества записей и их сортировка
posts_by_date_rdd_group = posts_in_period_.keyBy(lambda row: (row[0].year, row[1])).aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2).sortBy(lambda x: x[1], ascending=False).collect()

# Генерация списка годов
years = [i for i in range(2020, 2009, -1)]

# Формирование топа языков для каждого года
df_by_years = []
for year in years:
  df_by_years.extend([row for row in posts_by_date_rdd_group if row[0][0] == year][:10])

In [ ]:
# Формирование итогового датафрейма
row_template = Row('Year', 'Language', 'Count')
result_df = spark.createDataFrame([row_template(*x, y) for x, y in df_by_years])

# Сохранение отчета
result_df.write.parquet("top_langs.paquet")
result_df.show()

+----+----------+-----+
|Year|  Language|Count|
+----+----------+-----+
|2019|    Python|  162|
|2019|JavaScript|  131|
|2019|      Java|   95|
|2019|       PHP|   59|
|2019|         R|   36|
|2019|         C|   14|
|2019|      Dart|    9|
|2019|    MATLAB|    9|
|2019|        Go|    9|
|2019|      Bash|    8|
|2018|    Python|  214|
|2018|JavaScript|  196|
|2018|      Java|  145|
|2018|       PHP|   99|
|2018|         R|   63|
|2018|         C|   24|
|2018|     Scala|   22|
|2018|TypeScript|   21|
|2018|PowerShell|   13|
|2018|      Bash|   12|
+----+----------+-----+
only showing top 20 rows

